In [3]:
import os
import json
import gdown
import pandas as pd
import numpy as np
### Plot
import matplotlib.pyplot as plt
import seaborn as sns
### HTML
from bs4 import BeautifulSoup
import re
### Warnings
import warnings
### Text Preprocessing and Natural Language Processing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import nltk
import re
import spacy
from wordcloud import WordCloud

ModuleNotFoundError: No module named 'gdown'

In [ ]:
tmdb_df = pd.read_csv('TMDB_movie_dataset_v11.csv') #tmdb dataset we will be working with
tmdb_df
tmdb_df.describe()



NameError: name 'pd' is not defined

In [ ]:
# in order to create a robust key "title_year to join with the movielens dataset"
tmdb_df['release_date'] = pd.to_datetime(tmdb_df['release_date'], errors='coerce')
tmdb_df['year'] = tmdb_df['release_date'].dt.year.astype('Int64').astype(str).replace('nan', '')
tmdb_df['title_year'] = tmdb_df.apply(lambda x: f"{x['title']} ({x['year']})" if x['year'] else x['title'], axis=1)
tmdb_df


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,year,title_year
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili",2010,Inception (2010)
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,2014,Interstellar (2014)
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin",2008,The Dark Knight (2008)
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish",2009,Avatar (2009)
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian",2012,The Avengers (2012)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002262,664753,Dead Kennedys: Live in Vienna,0.000,0,Released,1982-01-01,0,16,False,NaN,...,Dead Kennedys filmed live at Arena Wien in Vie...,0.600,/nQ9RmZhdYhv8QHCoHgNBfyqewB6.jpg,NaN,Music,Welder,United States of America,English,1982,Dead Kennedys: Live in Vienna (1982)
1002263,664754,Haunted House,0.000,0,Released,2019-05-31,0,53,False,NaN,...,Three Paranormal Investigators spend the night...,0.600,/q72Scu36kQzoZX8TQ9vxnCLqDk1.jpg,NaN,Documentary,NaN,United States of America,English,2019,Haunted House (2019)
1002264,664755,"Fuck You, Purdue",0.000,0,Released,1987-07-07,0,12,False,NaN,...,A 1987 video by Cecilia Dougherty,0.600,NaN,NaN,NaN,NaN,NaN,NaN,1987,"Fuck You, Purdue (1987)"
1002265,664756,These Are the Rules,0.000,0,Released,1983-03-03,0,5,False,NaN,...,"In These Are the Rules, Doug Hall enacted an a...",0.600,NaN,NaN,NaN,NaN,NaN,NaN,1983,These Are the Rules (1983)


In [ ]:
df_movies = pd.read_csv('ml-1m/movies.csv', sep='::', engine='python', encoding='ISO-8859-1', header=None, names=['MovieID', 'Title', 'Genres']) #sohai give us 3 different files so need read them separately
print(df_movies.head())

   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy


In [ ]:
df_ratings = pd.read_csv('ml-1m/ratings.csv', sep='::', engine='python', encoding='ISO-8859-1', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp']) # ^same
print(df_ratings.head())

In [ ]:
df_users = pd.read_csv('ml-1m/users.csv', sep='::', engine='python', encoding='ISO-8859-1', header=None, names=['UserID', 'Gender', 'Age', 'Occupation','ZipCode']) # ^same
print(df_users.head())

In [ ]:
df_ml_combined = pd.merge(pd.merge(df_ratings, df_movies, on='MovieID'), df_users, on='UserID') #this is the merged version of the MovieLens Dataset
df_ml_combined

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,ZipCode
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067
...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,Weekend at Bernie's (1989),Comedy,M,25,6,11106
1000205,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War,M,25,6,11106
1000206,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama,M,25,6,11106
1000207,6040,1096,4,956715648,Sophie's Choice (1982),Drama,M,25,6,11106


In [ ]:
#now to finally merge the datasets
merged_df = pd.merge(df_ml_combined, tmdb_df, left_on='Title', right_on='title_year', how='left') #main dataset
merged_df


,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,ZipCode,...,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,year,title_year
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067,...,A petty criminal fakes insanity to serve his s...,36.673,/3jcbDmRFiQ83drXNOvRDeKHxS0C.jpg,"If he's crazy, what does that make you?",Drama,Fantasy Films,United States of America,English,1975,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067,...,When the young orphan boy James spills a magic...,27.497,/nl2oB6EbD1fHFuP2TLUHDtqs7Ux.jpg,Adventures this big don't grow on trees.,"Adventure, Animation, Family, Fantasy","Allied Filmmakers, Walt Disney Pictures",United States of America,English,1996,James and the Giant Peach (1996)
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067,...,A snobbish phonetics professor agrees to a wag...,19.547,/bTXVc29lGSNclf94VIZ49W4gGKl.jpg,The loverliest motion picture of them all!,"Romance, Drama, Comedy",Warner Bros. Pictures,United States of America,English,1964,My Fair Lady (1964)
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067,...,A twice-divorced mother of three who sees an i...,22.495,/xbMEHLu36LyptPD29FxVTndpYZ1.jpg,She brought a small town to its feet and a hug...,Drama,Jersey Films,United States of America,English,2000,Erin Brockovich (2000)
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012483,6040,1091,1,956716541,Weekend at Bernie's (1989),Comedy,M,25,6,11106,...,Two friends are invited for a weekend to a lux...,12.318,/a5FcWYJemR3jR8BrYihEorWRCDr.jpg,"Bernie may be dead, but he's still the life of...","Comedy, Crime","Gladden Entertainment, 20th Century Fox",United States of America,English,1989,Weekend at Bernie's (1989)
1012484,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War,M,25,6,11106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1012485,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama,M,25,6,11106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1012486,6040,1096,4,956715648,Sophie's Choice (1982),Drama,M,25,6,11106,...,"Stingo, a young writer, moves to Brooklyn in 1...",14.359,/6zayvLCWvF8ImK5UqwRUbETcpVU.jpg,"Between the innocent, the romantic, the sensua...","Drama, Romance, War","ITC Entertainment, Keith Barish Productions, U...","United Kingdom, United States of America","Czech, German, French, Polish, Russian, English",1982,Sophie's Choice (1982)
